In [ ]:
!pip install tavily-python requests beautifulsoup4 -q

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# TAVILY SOCIAL SCOUT + YUTORI TWITTER DEEP SCOUT
# Multi-Agent Viral Content Pipeline — Step 1b
#
# Run in Google Colab:
!pip install tavily-python requests -q
# ─────────────────────────────────────────────────────────────────────────────

import os
import json
import time
from getpass import getpass
from dataclasses import dataclass, field, asdict
from typing import List, Optional, Dict
from datetime import datetime
from collections import Counter

# ── Install ───────────────────────────────────────────────────────────────────
!pip install tavily-python requests -q

from tavily import TavilyClient
import requests


# ─────────────────────────────────────────────────────────────────────────────
# CONFIG — edit these
# ─────────────────────────────────────────────────────────────────────────────

TOPICS = [
    "OpenAI agents",
    "Claude AI",
    "Google Gemini",
    "AWS Bedrock",
    "GCP Vertex AI",
    "agentic AI platform",
]

PLATFORMS = {
    "twitter":   True,
    "linkedin":  True,
    "reddit":    True,
    "youtube":   True,
    "blogs":     True,
    "instagram": False,   # limited text content via Tavily
}

RECENCY      = "week"   # "day" | "week" | "month"
MAX_RESULTS  = 5       # results per query
USE_YUTORI   = True   # flip True + add Twitter Bearer Token for deep Twitter scrape


# ─────────────────────────────────────────────────────────────────────────────
# DATA STRUCTURES
# ─────────────────────────────────────────────────────────────────────────────

@dataclass
class ScoutedPost:
    platform:        str
    content_type:    str
    title:           str
    url:             str
    snippet:         str
    full_content:    str
    relevance_score: float
    topic:           str
    query_used:      str
    published_date:  Optional[str]       = None
    viral_signals:   List[str]           = field(default_factory=list)

@dataclass
class ScoutOutput:
    agent:             str       = "tavily_social_scout"
    timestamp:         str       = field(default_factory=lambda: datetime.utcnow().isoformat())
    topics_scouted:    List[str] = field(default_factory=list)
    platforms_scouted: List[str] = field(default_factory=list)
    posts:             List[ScoutedPost] = field(default_factory=list)
    total_found:       int       = 0
    queries_run:       int       = 0
    status:            str       = "success"


# ─────────────────────────────────────────────────────────────────────────────
# QUERY TEMPLATES — engineered to surface viral / high-engagement content
# ─────────────────────────────────────────────────────────────────────────────

QUERY_TEMPLATES = {

    "twitter": [
        "{topic} site:twitter.com viral thread 2025",
        "{topic} site:x.com trending",
        "{topic} CEO founder tweet thread site:twitter.com",
    ],

    "linkedin": [
        "{topic} site:linkedin.com CEO founder post viral",
        "{topic} linkedin.com/posts reactions comments",
        "{topic} site:linkedin.com 2025 trending",
    ],

    "reddit": [
        "{topic} site:reddit.com upvotes hot 2025",
        "{topic} reddit r/MachineLearning OR r/technology OR r/LocalLLaMA",
        "{topic} reddit r/artificial OR r/singularity discussion",
    ],

    "youtube": [
        "{topic} site:youtube.com review demo 2025",
        "{topic} youtube.com viral video breakdown",
        "{topic} CEO interview keynote youtube 2025",
    ],

    "blogs": [
        "{topic} site:substack.com 2025",
        "{topic} site:medium.com review analysis 2025",
        "{topic} developer review opinion breakdown 2025",
    ],
}

VIRAL_SIGNAL_WORDS = [
    "went viral", "blew up", "breaking", "just announced",
    "everyone is talking", "hot take", "thread", "repost",
    "comments", "likes", "upvotes", "views",
    "CEO", "founder", "leaked", "exclusive", "first look",
    "rant", "grief", "concern", "disappointed", "impressed",
]


# ─────────────────────────────────────────────────────────────────────────────
# HELPERS
# ─────────────────────────────────────────────────────────────────────────────

def detect_viral_signals(text: str) -> List[str]:
    text_lower = text.lower()
    return [s for s in VIRAL_SIGNAL_WORDS if s.lower() in text_lower]

def detect_platform(url: str, intended: str) -> str:
    u = url.lower()
    if "twitter.com" in u or "x.com" in u: return "twitter"
    if "linkedin.com"  in u:               return "linkedin"
    if "reddit.com"    in u:               return "reddit"
    if "youtube.com"   in u or "youtu.be" in u: return "youtube"
    if "substack.com"  in u:               return "substack"
    if "medium.com"    in u:               return "medium"
    if "instagram.com" in u:               return "instagram"
    return intended

def detect_content_type(url: str, title: str) -> str:
    u, t = url.lower(), title.lower()
    if "twitter.com" in u or "x.com" in u:
        return "thread" if "thread" in t else "post"
    if "youtube.com" in u or "youtu.be" in u: return "video"
    if "reddit.com"  in u:                    return "thread"
    if "linkedin.com" in u:                   return "post"
    if any(w in t for w in ["review","analysis","breakdown","rant"]): return "review"
    return "article"


# ─────────────────────────────────────────────────────────────────────────────
# TAVILY SOCIAL SCOUT
# ─────────────────────────────────────────────────────────────────────────────

class TavilySocialScout:

    def __init__(self, api_key: str):
        self.client = TavilyClient(api_key=api_key)

    def _run_query(self, query: str, topic: str, platform: str,
                   recency: str, max_results: int) -> List[ScoutedPost]:
        days_map = {"day": 2, "week": 7, "month": 30}
        try:
            resp = self.client.search(
                query=query,
                search_depth="advanced",
                max_results=max_results,
                topic="news",
                days=days_map.get(recency, 2),
                include_answer=False,
                include_raw_content=False,
            )
        except Exception:
            try:
                resp = self.client.search(
                    query=query,
                    search_depth="advanced",
                    max_results=max_results,
                    include_answer=False,
                )
            except Exception as e:
                print(f"      ⚠️  Query failed: {e}")
                return []

        posts = []
        for r in resp.get("results", []):
            url     = r.get("url", "")
            title   = r.get("title", "")
            content = r.get("content", "")
            posts.append(ScoutedPost(
                platform        = detect_platform(url, platform),
                content_type    = detect_content_type(url, title),
                title           = title,
                url             = url,
                snippet         = content[:400],
                full_content    = content,
                relevance_score = round(r.get("score", 0.0), 4),
                topic           = topic,
                query_used      = query,
                published_date  = r.get("published_date"),
                viral_signals   = detect_viral_signals(content),
            ))
        return posts

    def run(self, topics, platforms, recency="day", max_results=5) -> ScoutOutput:
        active = [p for p, on in platforms.items() if on]
        print(f"\n{'='*60}")
        print(f"📡 TAVILY SOCIAL SCOUT")
        print(f"{'='*60}")
        print(f"   Topics    : {topics}")
        print(f"   Platforms : {active}")
        print(f"   Recency   : last {recency}")

        output   = ScoutOutput(topics_scouted=topics, platforms_scouted=active)
        seen     = set()
        q_count  = 0

        for topic in topics:
            print(f"\n🔎 Topic: '{topic}'")
            for platform, enabled in platforms.items():
                if not enabled:
                    continue
                for template in QUERY_TEMPLATES.get(platform, [])[:2]:
                    query = template.replace("{topic}", topic)
                    print(f"   [{platform.upper():10s}] {query[:65]}")
                    posts = self._run_query(query, topic, platform, recency, max_results)
                    q_count += 1
                    new = 0
                    for p in posts:
                        if p.url not in seen:
                            seen.add(p.url)
                            output.posts.append(p)
                            new += 1
                    print(f"              → {new} new | {len(output.posts)} total")
                    time.sleep(0.3)

        # Rank: viral signal count first, then relevance score
        output.posts.sort(key=lambda p: (len(p.viral_signals), p.relevance_score), reverse=True)
        output.total_found = len(output.posts)
        output.queries_run = q_count

        print(f"\n{'='*60}")
        print(f"✅ SCOUT COMPLETE — {output.total_found} unique posts")
        plat_counts = Counter(p.platform for p in output.posts)
        for plat, cnt in plat_counts.most_common():
            print(f"   {plat:15s}: {cnt}")
        print(f"{'='*60}\n")
        return output


# ─────────────────────────────────────────────────────────────────────────────
# YUTORI / TWITTER v2 DEEP SCOUT (optional)
# ─────────────────────────────────────────────────────────────────────────────

def search_twitter_recent(query: str, bearer_token: str, max_results: int = 20) -> list:
    """
    Hits Twitter v2 recent search API.
    Returns tweets sorted by engagement (likes + retweets×3 + replies×2).
    """
    url     = "https://api.twitter.com/2/tweets/search/recent"
    headers = {"Authorization": f"Bearer {bearer_token}"}
    params  = {
        "query":        f"{query} -is:retweet lang:en",
        "max_results":  min(max_results, 100),
        "tweet.fields": "public_metrics,created_at,author_id,entities",
        "expansions":   "author_id",
        "user.fields":  "name,username,verified,public_metrics",
    }

    resp = requests.get(url, headers=headers, params=params)
    resp.raise_for_status()
    data  = resp.json()
    tweets = data.get("data", [])
    users  = {u["id"]: u for u in data.get("includes", {}).get("users", [])}

    results = []
    for t in tweets:
        m      = t.get("public_metrics", {})
        author = users.get(t.get("author_id", ""), {})
        results.append({
            "tweet_id":         t["id"],
            "url":              f"https://twitter.com/{author.get('username','')}/status/{t['id']}",
            "text":             t["text"],
            "author":           author.get("name", ""),
            "username":         author.get("username", ""),
            "verified":         author.get("verified", False),
            "author_followers": author.get("public_metrics", {}).get("followers_count", 0),
            "likes":            m.get("like_count", 0),
            "retweets":         m.get("retweet_count", 0),
            "replies":          m.get("reply_count", 0),
            "engagement":       m.get("like_count", 0) + m.get("retweet_count", 0) * 3 + m.get("reply_count", 0) * 2,
            "created_at":       t.get("created_at", ""),
        })

    results.sort(key=lambda x: x["engagement"], reverse=True)
    return results


def run_yutori_scout(topics: List[str], bearer_token: str, max_results: int = 10) -> list:
    all_tweets = []
    for topic in topics:
        print(f"🐦 Twitter: '{topic}'")
        tweets = search_twitter_recent(topic, bearer_token, max_results)
        all_tweets.extend(tweets)
        for t in tweets[:3]:
            print(f"   ❤️ {t['likes']:5d}  🔁 {t['retweets']:4d}  💬 {t['replies']:4d} | @{t['username']}: {t['text'][:80]}")
    all_tweets.sort(key=lambda x: x["engagement"], reverse=True)
    return all_tweets


# ─────────────────────────────────────────────────────────────────────────────
# DISPLAY HELPERS
# ─────────────────────────────────────────────────────────────────────────────

PLATFORM_EMOJI = {
    "twitter": "🐦", "linkedin": "💼", "reddit": "🤖",
    "youtube": "📹", "substack": "📝", "medium": "📰",
    "blog": "🌐",   "instagram": "📸",
}
TYPE_EMOJI = {
    "thread": "🧵", "post": "📌", "video": "🎬",
    "article": "📄", "review": "⭐",
}

def print_leaderboard(output: ScoutOutput, limit: int = 20):
    print(f"\n🔥 VIRAL CONTENT LEADERBOARD — Top {min(limit, output.total_found)}\n")
    print("─" * 70)
    for i, p in enumerate(output.posts[:limit]):
        pe      = PLATFORM_EMOJI.get(p.platform, "🌐")
        te      = TYPE_EMOJI.get(p.content_type, "📄")
        signals = ", ".join(p.viral_signals[:4]) or "none"
        print(f"#{i+1:02d} {pe} [{p.platform.upper()}] {te} {p.content_type.upper()}")
        print(f"     📰 {p.title[:70]}")
        print(f"     🔗 {p.url}")
        print(f"     🏷️  {p.topic}  |  📊 {p.relevance_score}  |  🔥 {signals}")
        if p.published_date:
            print(f"     📅 {p.published_date}")
        print(f"     💬 {p.snippet[:180]}...")
        print("─" * 70)

def show_platform(output: ScoutOutput, platform_name: str, limit: int = 5):
    filtered = [p for p in output.posts if p.platform == platform_name]
    print(f"\n{PLATFORM_EMOJI.get(platform_name,'🌐')} {platform_name.upper()} — {len(filtered)} results\n")
    for p in filtered[:limit]:
        print(f"  📰 {p.title}")
        print(f"  🔗 {p.url}")
        print(f"  🔥 {', '.join(p.viral_signals[:5]) or 'no signals'}")
        print(f"  💬 {p.snippet[:200]}\n")


# ─────────────────────────────────────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────────────────────────────────────

if __name__ == "__main__":

    # ── API keys ──────────────────────────────────────────────
    TAVILY_API_KEY = getpass("Tavily API key: ")

    # ── Run Tavily social scout ───────────────────────────────
    scout        = TavilySocialScout(api_key=TAVILY_API_KEY)
    scout_output = scout.run(
        topics      = TOPICS,
        platforms   = PLATFORMS,
        recency     = RECENCY,
        max_results = MAX_RESULTS,
    )

    # ── Print leaderboard ─────────────────────────────────────
    print_leaderboard(scout_output, limit=20)

    # ── Per-platform views ────────────────────────────────────
    for plat in ["twitter", "reddit", "youtube"]:
        show_platform(scout_output, plat, limit=3)

    # ── Save outputs ──────────────────────────────────────────
    with open("tavily_scout_output.json", "w") as f:
        json.dump(asdict(scout_output), f, indent=2, default=str)

    url_list = [
        {
            "rank":          i + 1,
            "platform":      p["platform"],
            "content_type":  p["content_type"],
            "title":         p["title"],
            "url":           p["url"],
            "score":         p["relevance_score"],
            "viral_signals": p["viral_signals"],
            "topic":         p["topic"],
            "published_date":p["published_date"],
        }
        for i, p in enumerate(asdict(scout_output)["posts"])
    ]
    with open("tavily_scout_urls.json", "w") as f:
        json.dump(url_list, f, indent=2)

    print("✅ Saved: tavily_scout_output.json + tavily_scout_urls.json")

    # ── Optional Yutori / Twitter v2 deep scout ───────────────
    if USE_YUTORI:
        TWITTER_BEARER = getpass("Twitter Bearer Token: ")
        twitter_results = run_yutori_scout(TOPICS, TWITTER_BEARER, max_results=10)
        with open("yutori_twitter_output.json", "w") as f:
            json.dump(twitter_results, f, indent=2)
        print(f"✅ Saved {len(twitter_results)} tweets → yutori_twitter_output.json")
    else:
        print("ℹ️  Yutori disabled. Set USE_YUTORI = True to enable.")

Tavily API key: ··········

📡 TAVILY SOCIAL SCOUT
   Topics    : ['OpenAI agents', 'Claude AI', 'Google Gemini', 'AWS Bedrock', 'GCP Vertex AI', 'agentic AI platform']
   Platforms : ['twitter', 'linkedin', 'reddit', 'youtube', 'blogs']
   Recency   : last day

🔎 Topic: 'OpenAI agents'
   [TWITTER   ] OpenAI agents site:twitter.com viral thread 2025


/tmp/ipython-input-880/3653994553.py:73: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp:         str       = field(default_factory=lambda: datetime.utcnow().isoformat())


              → 5 new | 5 total
   [TWITTER   ] OpenAI agents site:x.com trending
              → 5 new | 10 total
   [LINKEDIN  ] OpenAI agents site:linkedin.com CEO founder post viral
              → 2 new | 12 total
   [LINKEDIN  ] OpenAI agents linkedin.com/posts reactions comments
              → 5 new | 17 total
   [REDDIT    ] OpenAI agents site:reddit.com upvotes hot 2025
              → 5 new | 22 total
   [REDDIT    ] OpenAI agents reddit r/MachineLearning OR r/technology OR r/Local
              → 3 new | 25 total
   [YOUTUBE   ] OpenAI agents site:youtube.com review demo 2025
              → 5 new | 30 total
   [YOUTUBE   ] OpenAI agents youtube.com viral video breakdown
              → 4 new | 34 total
   [BLOGS     ] OpenAI agents site:substack.com 2025
              → 5 new | 39 total
   [BLOGS     ] OpenAI agents site:medium.com review analysis 2025
              → 5 new | 44 total

🔎 Topic: 'Claude AI'
   [TWITTER   ] Claude AI site:twitter.com viral thread 2025
      

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# TAVILY SCOUT FILTER + SCORER
# Multi-Agent Viral Content Pipeline — Step 1c
#
# Reads:  tavily_scout_output.json  (223 posts from Step 1b)
# Writes: scout_filtered.json       (top N posts — Reka-ready)
#         scout_filtered_urls.json  (clean URL list)
#
# Scoring logic:
#   - Engagement signals (likes, views, comments, retweets in snippet)
#   - Recency boost (last 24h > last 48h > older)
#   - Platform weight (twitter/reddit > youtube > linkedin > blog)
#   - Content type weight (thread/video/rant > article)
#   - Controversy signals (grief, rant, concern, disappointed)
#   - Hype signals (insane, viral, breaking, exclusive, first look)
#   - Author authority (CEO, founder, verified)
# ─────────────────────────────────────────────────────────────────────────────

import json
import re
from datetime import datetime, timezone, timedelta
from dataclasses import dataclass, field, asdict
from typing import List, Optional, Tuple
from collections import defaultdict


# ─────────────────────────────────────────────────────────────────────────────
# CONFIG
# ─────────────────────────────────────────────────────────────────────────────

INPUT_FILE          = "tavily_scout_output.json"
OUTPUT_FILE         = "scout_filtered.json"
OUTPUT_URLS_FILE    = "scout_filtered_urls.json"

TOP_N               = 25     # how many posts to pass to Reka
MIN_SCORE           = 0.10   # drop anything below this threshold

# How many posts per platform max (prevents one platform flooding Reka)
MAX_PER_PLATFORM    = 8

# Which content types to prioritize for VIDEO generation
PREFERRED_TYPES     = ["video", "thread", "review", "post", "article"]


# ─────────────────────────────────────────────────────────────────────────────
# SCORING WEIGHTS
# ─────────────────────────────────────────────────────────────────────────────

PLATFORM_WEIGHT = {
    "twitter":   1.4,
    "reddit":    1.3,
    "youtube":   1.2,
    "substack":  1.0,
    "medium":    0.9,
    "linkedin":  0.8,
    "instagram": 0.6,
}

CONTENT_TYPE_WEIGHT = {
    "video":    1.3,
    "thread":   1.2,
    "review":   1.1,
    "post":     1.0,
    "article":  0.9,
}

# Signals and their score contribution
HYPE_SIGNALS = {
    "went viral":    0.20,
    "breaking":      0.15,
    "exclusive":     0.12,
    "first look":    0.12,
    "just announced":0.10,
    "insane":        0.10,
    "impressed":     0.08,
}

CONTROVERSY_SIGNALS = {
    "grief":         0.18,
    "rant":          0.16,
    "concern":       0.12,
    "disappointed":  0.12,
    "hot take":      0.10,
    "everyone is talking": 0.10,
}

AUTHORITY_SIGNALS = {
    "CEO":           0.15,
    "founder":       0.12,
    "verified":      0.10,
    "leaked":        0.14,
    "president":     0.10,
}

# Regex patterns to extract raw engagement numbers from snippet text
ENGAGEMENT_PATTERNS = {
    "views":    [
        r"([\d,\.]+[KkMm]?)\s*views",
        r"([\d,\.]+[KkMm]?)\s*view",
    ],
    "likes":    [
        r"([\d,\.]+[KkMm]?)\s*likes",
        r"([\d,\.]+[KkMm]?)\s*like",
    ],
    "comments": [
        r"([\d,\.]+[KkMm]?)\s*[Cc]omments",
        r"([\d,\.]+[KkMm]?)\s*replies",
    ],
    "subs": [
        r"([\d,\.]+[KkMm]?)\s*subscribers",
    ],
    "reposts": [
        r"([\d,\.]+[KkMm]?)\s*reposts?",
        r"([\d,\.]+[KkMm]?)\s*retweets?",
    ],
}


# ─────────────────────────────────────────────────────────────────────────────
# HELPERS
# ─────────────────────────────────────────────────────────────────────────────

def parse_num(s: str) -> float:
    """Convert '4.9K', '1.6M', '139K' → float."""
    s = s.replace(",", "").strip()
    multipliers = {"k": 1_000, "m": 1_000_000, "b": 1_000_000_000}
    if s and s[-1].lower() in multipliers:
        return float(s[:-1]) * multipliers[s[-1].lower()]
    try:
        return float(s)
    except ValueError:
        return 0.0

def extract_engagement(text: str) -> dict:
    """Pull raw numbers from snippet text. Returns dict of metric → value."""
    metrics = {}
    for metric, patterns in ENGAGEMENT_PATTERNS.items():
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                metrics[metric] = parse_num(match.group(1))
                break
    return metrics

def engagement_score(metrics: dict) -> float:
    """Normalize engagement numbers into a 0–1 score."""
    views    = metrics.get("views", 0)
    likes    = metrics.get("likes", 0)
    comments = metrics.get("comments", 0)
    reposts  = metrics.get("reposts", 0)
    subs     = metrics.get("subs", 0)

    # Raw engagement score (weighted sum)
    raw = (
        views    * 1.0 +
        likes    * 3.0 +
        comments * 4.0 +
        reposts  * 5.0 +
        subs     * 0.1
    )
    # Normalize: cap at 10M, log scale
    import math
    if raw <= 0:
        return 0.0
    return min(math.log10(raw + 1) / 7.0, 1.0)   # log10(10M) ≈ 7

def recency_score(published_date: Optional[str]) -> float:
    """Return 1.0 if today, 0.7 if yesterday, 0.4 if older, 0.2 if unknown."""
    if not published_date:
        return 0.2
    try:
        # Handle RFC 2822 and ISO formats
        for fmt in [
            "%a, %d %b %Y %H:%M:%S %Z",
            "%a, %d %b %Y %H:%M:%S %z",
            "%Y-%m-%dT%H:%M:%S",
            "%Y-%m-%dT%H:%M:%SZ",
            "%Y-%m-%d",
        ]:
            try:
                dt = datetime.strptime(published_date.strip(), fmt)
                if dt.tzinfo is None:
                    dt = dt.replace(tzinfo=timezone.utc)
                break
            except ValueError:
                continue
        else:
            return 0.2

        now   = datetime.now(timezone.utc)
        delta = now - dt

        if delta < timedelta(hours=24):  return 1.0
        if delta < timedelta(hours=48):  return 0.7
        if delta < timedelta(days=7):    return 0.4
        return 0.2

    except Exception:
        return 0.2


# ─────────────────────────────────────────────────────────────────────────────
# SCORER
# ─────────────────────────────────────────────────────────────────────────────

@dataclass
class ScoredPost:
    # Original fields
    platform:        str
    content_type:    str
    title:           str
    url:             str
    snippet:         str
    full_content:    str
    relevance_score: float
    topic:           str
    published_date:  Optional[str]
    viral_signals:   List[str]

    # Computed scores
    final_score:        float = 0.0
    engagement_score:   float = 0.0
    recency_score:      float = 0.0
    signal_score:       float = 0.0
    authority_score:    float = 0.0
    platform_weight:    float = 1.0
    content_type_weight:float = 1.0
    engagement_metrics: dict  = field(default_factory=dict)
    score_breakdown:    dict  = field(default_factory=dict)


def score_post(raw: dict) -> ScoredPost:
    text = (raw.get("title", "") + " " + raw.get("snippet", "") + " " + raw.get("full_content", ""))

    # Extract engagement numbers
    eng_metrics = extract_engagement(text)
    eng_s       = engagement_score(eng_metrics)

    # Recency
    rec_s = recency_score(raw.get("published_date"))

    # Hype signals
    hype_s = sum(
        weight for signal, weight in HYPE_SIGNALS.items()
        if signal.lower() in text.lower()
    )

    # Controversy signals
    cont_s = sum(
        weight for signal, weight in CONTROVERSY_SIGNALS.items()
        if signal.lower() in text.lower()
    )

    # Authority signals
    auth_s = sum(
        weight for signal, weight in AUTHORITY_SIGNALS.items()
        if signal.lower() in text.lower()
    )

    signal_s    = min(hype_s + cont_s, 1.0)
    platform_w  = PLATFORM_WEIGHT.get(raw.get("platform", ""), 1.0)
    type_w      = CONTENT_TYPE_WEIGHT.get(raw.get("content_type", ""), 1.0)
    tavily_s    = raw.get("relevance_score", 0.0)

    # Final weighted score
    final = (
        eng_s       * 0.30 +
        rec_s       * 0.20 +
        signal_s    * 0.20 +
        tavily_s    * 0.15 +
        auth_s      * 0.15
    ) * platform_w * type_w

    return ScoredPost(
        platform         = raw.get("platform", ""),
        content_type     = raw.get("content_type", ""),
        title            = raw.get("title", ""),
        url              = raw.get("url", ""),
        snippet          = raw.get("snippet", ""),
        full_content     = raw.get("full_content", ""),
        relevance_score  = raw.get("relevance_score", 0.0),
        topic            = raw.get("topic", ""),
        published_date   = raw.get("published_date"),
        viral_signals    = raw.get("viral_signals", []),
        final_score      = round(final, 4),
        engagement_score = round(eng_s, 4),
        recency_score    = round(rec_s, 4),
        signal_score     = round(signal_s, 4),
        authority_score  = round(auth_s, 4),
        platform_weight  = platform_w,
        content_type_weight = type_w,
        engagement_metrics  = eng_metrics,
        score_breakdown  = {
            "engagement":   round(eng_s * 0.30, 4),
            "recency":      round(rec_s * 0.20, 4),
            "signals":      round(signal_s * 0.20, 4),
            "tavily":       round(tavily_s * 0.15, 4),
            "authority":    round(auth_s * 0.15, 4),
            "platform_mul": platform_w,
            "type_mul":     type_w,
        },
    )


# ─────────────────────────────────────────────────────────────────────────────
# FILTER + RANK
# ─────────────────────────────────────────────────────────────────────────────

def filter_and_rank(
    posts: list,
    top_n: int             = TOP_N,
    min_score: float       = MIN_SCORE,
    max_per_platform: int  = MAX_PER_PLATFORM,
) -> Tuple[List[ScoredPost], dict]:

    # Score all
    scored = [score_post(p) for p in posts]

    # Drop below threshold
    scored = [p for p in scored if p.final_score >= min_score]

    # Sort by final score
    scored.sort(key=lambda p: p.final_score, reverse=True)

    # Per-platform cap to ensure diversity
    platform_counts = defaultdict(int)
    diverse = []
    for p in scored:
        if platform_counts[p.platform] < max_per_platform:
            diverse.append(p)
            platform_counts[p.platform] += 1

    top = diverse[:top_n]

    stats = {
        "total_input":    len(posts),
        "after_threshold":len(scored),
        "after_diversity": len(diverse),
        "final_selected": len(top),
        "platform_distribution": dict(platform_counts),
        "score_range": {
            "max": round(top[0].final_score, 4) if top else 0,
            "min": round(top[-1].final_score, 4) if top else 0,
        },
    }
    return top, stats


# ─────────────────────────────────────────────────────────────────────────────
# DISPLAY
# ─────────────────────────────────────────────────────────────────────────────

PLATFORM_EMOJI = {
    "twitter": "🐦", "linkedin": "💼", "reddit": "🤖",
    "youtube": "📹", "substack": "📝", "medium": "📰",
    "blog": "🌐",   "instagram": "📸",
}
TYPE_EMOJI = {
    "thread": "🧵", "post": "📌", "video": "🎬",
    "article": "📄", "review": "⭐",
}

def print_filtered(posts: List[ScoredPost], stats: dict):
    print(f"\n{'='*65}")
    print(f"🎯 FILTERED LEADERBOARD — Top {stats['final_selected']} posts for Reka")
    print(f"{'='*65}")
    print(f"   Input   : {stats['total_input']} posts")
    print(f"   Dropped : {stats['total_input'] - stats['after_threshold']} below threshold")
    print(f"   Selected: {stats['final_selected']}")
    print(f"   Score range: {stats['score_range']['min']} – {stats['score_range']['max']}")
    print(f"\n   Platform mix:")
    for plat, cnt in sorted(stats['platform_distribution'].items(), key=lambda x: -x[1]):
        print(f"     {PLATFORM_EMOJI.get(plat,'🌐')} {plat:12s}: {cnt}")
    print()

    for i, p in enumerate(posts):
        pe = PLATFORM_EMOJI.get(p.platform, "🌐")
        te = TYPE_EMOJI.get(p.content_type, "📄")

        # Build engagement string
        eng_parts = []
        m = p.engagement_metrics
        if m.get("views"):    eng_parts.append(f"👁 {int(m['views']):,}")
        if m.get("likes"):    eng_parts.append(f"❤️ {int(m['likes']):,}")
        if m.get("comments"): eng_parts.append(f"💬 {int(m['comments']):,}")
        if m.get("reposts"):  eng_parts.append(f"🔁 {int(m['reposts']):,}")
        if m.get("subs"):     eng_parts.append(f"👤 {int(m['subs']):,} subs")
        eng_str = "  ".join(eng_parts) if eng_parts else "no metrics extracted"

        print(f"#{i+1:02d} {'█' * int(p.final_score * 20):<20} {p.final_score:.3f}")
        print(f"    {pe} {p.platform.upper():10s}  {te} {p.content_type.upper()}")
        print(f"    📰 {p.title[:72]}")
        print(f"    🔗 {p.url}")
        print(f"    🏷️  {p.topic}")
        if eng_str != "no metrics extracted":
            print(f"    📊 {eng_str}")
        if p.published_date:
            print(f"    📅 {p.published_date[:30]}")
        print(f"    🔥 signals: {', '.join(p.viral_signals[:4]) or 'none'}")
        print(f"    📈 breakdown → eng:{p.score_breakdown['engagement']:.3f} "
              f"rec:{p.score_breakdown['recency']:.3f} "
              f"sig:{p.score_breakdown['signals']:.3f} "
              f"auth:{p.score_breakdown['authority']:.3f}")
        print(f"    💬 {p.snippet[:160]}...")
        print()


# ─────────────────────────────────────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────────────────────────────────────

if __name__ == "__main__":

    # ── Load scout output ─────────────────────────────────────
    print(f"📂 Loading {INPUT_FILE}...")
    with open(INPUT_FILE, "r") as f:
        raw_data = json.load(f)

    posts = raw_data.get("posts", [])
    print(f"   {len(posts)} posts loaded")
    print(f"   Topics: {raw_data.get('topics_scouted', [])}")
    print(f"   Scouted: {raw_data.get('timestamp', '')[:19]}")

    # ── Score + filter ────────────────────────────────────────
    print(f"\n⚙️  Scoring all {len(posts)} posts...")
    top_posts, stats = filter_and_rank(
        posts,
        top_n            = TOP_N,
        min_score        = MIN_SCORE,
        max_per_platform = MAX_PER_PLATFORM,
    )

    # ── Display ───────────────────────────────────────────────
    print_filtered(top_posts, stats)

    # ── Save full filtered output (for Reka) ──────────────────
    output = {
        "agent":            "scout_filter",
        "timestamp":        datetime.now(timezone.utc).isoformat(),
        "source_file":      INPUT_FILE,
        "filter_config": {
            "top_n":            TOP_N,
            "min_score":        MIN_SCORE,
            "max_per_platform": MAX_PER_PLATFORM,
        },
        "stats":    stats,
        "posts":    [asdict(p) for p in top_posts],
    }

    with open(OUTPUT_FILE, "w") as f:
        json.dump(output, f, indent=2, default=str)

    # ── Save clean URL list ───────────────────────────────────
    url_list = [
        {
            "rank":             i + 1,
            "final_score":      p.final_score,
            "platform":         p.platform,
            "content_type":     p.content_type,
            "title":            p.title,
            "url":              p.url,
            "topic":            p.topic,
            "published_date":   p.published_date,
            "viral_signals":    p.viral_signals,
            "engagement":       p.engagement_metrics,
            "score_breakdown":  p.score_breakdown,
            "snippet":          p.snippet[:300],
        }
        for i, p in enumerate(top_posts)
    ]

    with open(OUTPUT_URLS_FILE, "w") as f:
        json.dump(url_list, f, indent=2, default=str)

    print(f"\n✅ Saved:")
    print(f"   {OUTPUT_FILE}       → Reka Agent (Step 2) full input")
    print(f"   {OUTPUT_URLS_FILE}  → ranked URL list")
    print(f"\n   Feed '{OUTPUT_FILE}' into Reka Agent as tavily_data.")
    print(f"\n{'='*65}")
    print(f"Pipeline:")
    print(f"  ✅ Tavily Scout (223 posts)")
    print(f"  ✅ Scout Filter ({len(top_posts)} posts → Reka-ready)")
    print(f"  🧠 Reka Agent  ← next")
    print(f"{'='*65}")

📂 Loading tavily_scout_output.json...
   226 posts loaded
   Topics: ['OpenAI agents', 'Claude AI', 'Google Gemini', 'AWS Bedrock', 'GCP Vertex AI', 'agentic AI platform']
   Scouted: 2026-02-27T22:13:07

⚙️  Scoring all 226 posts...

🎯 FILTERED LEADERBOARD — Top 25 posts for Reka
   Input   : 226 posts
   Dropped : 20 below threshold
   Selected: 25
   Score range: 0.2341 – 0.6949

   Platform mix:
     📹 youtube     : 8
     🐦 twitter     : 8
     🤖 reddit      : 8
     💼 linkedin    : 8
     📝 substack    : 8
     📰 medium      : 8

#01 █████████████        0.695
    📹 YOUTUBE     🎬 VIDEO
    📰 Introduction to Operator & Agents - YouTube
    🔗 https://www.youtube.com/watch?v=CSE77wAdDLg
    🏷️  OpenAI agents
    📊 👁 863,638  💬 1,809  👤 1,910,000 subs
    🔥 signals: comments, views
    📈 breakdown → eng:0.258 rec:0.040 sig:0.000 auth:0.000
    💬 Introduction to Operator & Agents
OpenAI
1910000 subscribers

863638 views
23 Jan 2025
Begins at 10am PT

Join Sam Altman, Yash Kumar, Casey